# Signifinace Tests

A significance test answers the question: what is the probability that a sample comes from a population. If the probability is low we can conclude that it is unlikely that the sample belongs to that population. However, if the probability is high it doesn't mean it is from the same population, we just didn't prove it isn't. We can never prove it is.
By convention the following probability levels were chosen:

* \> 5% nlikelyhood that the sample could have been drawn by chance from the population is high and therefore we can't say anything about the relationship between the 2
* 1% < x < 5% - the sample probably doesn't belong to population
* < 1% - the sample is very likely to belong to another population 

**Based on "Practical Statistics - Simply Explained" by Russel Langley**

## zM Test (1733)

In [1]:
zm <- function(sample, population_mean, population_sd, sides=2) {
    result <- abs(mean(sample) - population_mean) * sqrt(length(sample)) / population_sd
    return(100 * pnorm(result, lower.tail=FALSE) * sides)
}
# note the use of 2-sided probabilities by default because we ask whether there is a
# difference in result, not whether the difference is smaller or greater 
# (we use absolute value calculation) 

In [2]:
# Problem 1: 
# Gold melts at 1060, assume a sd of 3 degrees obtained by running many melting experiments.
# An unknown metal was found out to melt at 1072. 
# What is the probability that the metal is gold ?

cat(sprintf('Prob(metal=gold)=%.6f%%', zm(1072, 1060, 3)))

Prob(metal=gold)=0.006334%

In [3]:
# Problem 2:
# 40 patients with a certain disease have a mean pulse rate of 73
# Mean pulse rate for healthy people = 70 with an sd = 5
# How likely the increased pulse rate is due to illness and not just a random variation of a set of healthy people ?

cat(sprintf('Prob(disease causes higher pulse rate)=%.6f%%', zm(rep(73, 40), 70, 5)))

# Pulse rate is statistically significant but not practically significant (only 3 extra beats per minute).

Prob(disease causes higher pulse rate)=0.014780%

In [4]:
# Problem 3:
# Copper melts at 1080 degrees with an sd = 5.
# How likely that metal from Problem 1 is copper ?

cat(sprintf('Prob(metal=copper)=%.6f%%\n', zm(1072, 1080, 5)))

# and if we retest the melting temperature of the unknown metal ?
cat(sprintf('Prob(metal=copper)=%.6f%%', zm(c(1072, 1071, 1072, 1073), 1080, 5)))

Prob(metal=copper)=10.959858%
Prob(metal=copper)=0.137428%

## Student's _t_ Test (1908)

In [5]:
# same formula as for zM test with 2 exceptions:
# 1. The standard deviation of the sample is used (sd of the population is unknown)
# 2. Obtaining the p-value from the result depends on the sample size (so is like p-values for z 
# but different values, depending on sample size -> smaller the sample the larger the z value for 
# the same p-value)

# Problem 1:
# A printing press can make 45 copies per minute
# After an adjustment to increase its throughput we obtain: 46, 47, 48 copies per minute.
# How likely the change is due to random variation rather than the effect of adjustment ?

result <- t.test(x=c(46, 47, 48), mu=45, conf.level=0.95);
cat(
    sprintf(
        'Possible values for the mean of a 3-item sample with confidence interval %s %%: [%s]\np-value for current sample: %s\n', 
        attr(result$conf.int, 'conf.level'),
        toString(result$conf.int),
        result$p.value
    )
)

Possible values for the mean of a 3-item sample with confidence interval 0.95 %: [44.5158622882497, 49.4841377117503]
p-value for current sample: 0.0741799002274485


In [6]:
# Problem 2:
# If the next 2 trials in Problem 1 yield: 47, 47 how is the p-value changing ?
result <- t.test(x=c(46, 47, 48, 47, 47), mu=45, conf.level=0.95);
cat(
    sprintf(
        'Possible values for the mean for the 5-item sample with confidence interval %s %%: [%s]\np-value for current sample: %s\n', 
        attr(result$conf.int, 'conf.level'),
        toString(result$conf.int),
        result$p.value
    )
)

# note the second confidence interval does not intersect with the true mean -> variation is very likely
# due to improvement rather than chance.

Possible values for the mean for the 5-item sample with confidence interval 0.95 %: [46.1220109669149, 47.8779890330851]
p-value for current sample: 0.00319820215233531


In [7]:
# Problem 3:
# Between 1945 and 1962 the telephone bill of a CEO averaged $48 per year. 
# With the new secretary the values were: $56, $51, $63, $60
# Should the secretary be fired for over-using the phone ?
result <- t.test(x=c(56, 51, 63, 60), mu=48, conf.level=0.95);
cat(
    sprintf(
        'Possible values for the mean for the 4-item sample with confidence interval %s %%: [%s]\np-value for current sample: %s\n', 
        attr(result$conf.int, 'conf.level'),
        toString(result$conf.int),
        result$p.value
    )
)
# Probably yes, unless the phone service became more expensive on its own.

Possible values for the mean for the 4-item sample with confidence interval 0.95 %: [49.2317619603331, 65.7682380396669]
p-value for current sample: 0.0353301023027491


In [8]:
# Problem 4:
# The average income of a popultion is $20560
# A random sample of 49 people from a neighbour region is 18505
# Is the difference statistically significant ?

# Not enough info -> standard deviation of the sample is not known.

sample1 <- seq(from=18505-49, to=18505+49, by=2)
result <- t.test(x=sample1, mu=20560, conf.level=0.95);
cat(
    sprintf(
        'Everyone got the almost the same income [%s, %s] (sd=%s): confidence interval %s %%: [%s]\np-value for current sample: %s\n', 
        min(sample1), max(sample1),
        sd(sample1),
        attr(result$conf.int, 'conf.level'),
        toString(result$conf.int),
        result$p.value
    )
)

sample2 <- seq(from=18505-49*300, to=18505+49*300, by=600)
result <- t.test(x=sample2, mu=20560, conf.level=0.95);
cat(
    sprintf(
        'Everyone got the different incomes [%s, %s] (sd=%s): confidence interval %s %%: [%s]\np-value for current sample: %s\n', 
        min(sample2), max(sample2),
        sd(sample2),
        attr(result$conf.int, 'conf.level'),
        toString(result$conf.int),
        result$p.value
    )
)

Everyone got the almost the same income [18456, 18554] (sd=29.1547594742265): confidence interval 0.95 %: [18496.7143090347, 18513.2856909653]
p-value for current sample: 1.90805983449625e-92
Everyone got the different incomes [3805, 33205] (sd=8746.42784226795): confidence interval 0.95 %: [16019.2927104071, 20990.7072895929]
p-value for current sample: 0.103025307777696


## Wilcoxon's Sum of Ranks Test (1945)

In [9]:
# Compare 2 samples of measurements with the assumption that they come from 2 populations with equal means.
# Instead of using the values of measurements we rank them in order and based on the smallest
# sum of ranks determine the probability of obtaining that sum due to chance.

# Instead of determining the probability of the parent groups having the same mean, we could determine
# the probability of the sample groups having the same standard deviation. For that we need to rank differently.
# We use the Siegel and Tukey's "paired alternation" of ranking: 
# * smallest measurement - rank 1
# * largest measurement and second largest - rank 2 and 3
# * second and third smallest measurements - rank 4 and 5
# * ...
# This way the extreme values get the smaller ranks. To increase the accuracy of the Siegel-Tukey test
# you need to make the means of the samples equal (e.g. add the difference between the means to the sample with smaller mean).

# The test can be applied for any probability distribution.

# Ranking should be made in both directions (smallest to largest and largest to smallest)
# if number of measurements in the 2 samples are not equal and both are smaller than 20.
# You need to pick the smallest sum of ranks from the 4 values. Otherwise ranking in a single direction is sufficient.

In [10]:
# Problem 1:
# Examine whether there is a significant different in the recovery times of 2 groups of patients
# on which a specific operation to remove the gallbladder is performed.

group1 <- c(16,20,25,19,22,15,22,19)
group2 <- c(18,19,15,16,21,17,17,14)

w <- wilcox.test(group1, group2, exact=FALSE, conf.int=0.95)  # exact=FALSE turns off the warning when measurements with equal values are present
# in general ties make the test less accurate.

cat(
    sprintf(
        'There is no significant difference in operations on patient recovery: %.4f,\nmean1=%.4f, mean2=%.4f', 
        w$p.value, 
        mean(group1),
        mean(group2)
    )
)


There is no significant difference in operations on patient recovery: 0.1015,
mean1=19.7500, mean2=17.1250

In [11]:
# Problem 2:
# A man recorded the number of advertisments in 2 radio stations. Is there sufficient evidence that
# one station airs more ads than the other ?
station1 <- c(341, 326, 360, 305, 326)
station2 <- c(352, 382, 347)

w <- wilcox.test(station1, station2, exact=FALSE)
cat(
    sprintf(
        'There is no significant difference in number of ads between the stations: %.4f,\nmean1=%.4f, mean2=%.4f', 
        w$p.value, 
        mean(station1),
        mean(station2)
    )
)

cat(sprintf('\n\n'))

t <- t.test(x=station1, y=station2, mu=0)  # mu is the difference of means -> i.e. no difference

cat(
    sprintf(
        'There is no significant difference in number of ads between the stations: %.4f,\nmean1=%.4f, mean2=%.4f', 
        t$p.value, 
        mean(station1),
        mean(station2)
    )
)

cat(sprintf('\nThe value of the t-test is more precise but does not change the overall conclusion.'))


There is no significant difference in number of ads between the stations: 0.1337,
mean1=331.6000, mean2=360.3333

There is no significant difference in number of ads between the stations: 0.1041,
mean1=331.6000, mean2=360.3333
The value of the t-test is more precise but does not change the overall conclusion.

In [12]:
# Perform the Siegel-Tukey test on data in Problem 2 to determine whether there is a significant
# difference in dispersions of the samples.

if(!require(jmuOutlier)) {
    install.packages("jmuOutlier"); require(jmuOutlier)
}

siegel_tukey <- function(x, y){
    m1 <- mean(x)
    m2 <- mean(y)
    if (m1 < m2){
        x <- x + (m2 - m1)
    }
    else {
        y <- y + (m2 - m1)
    }
    if (length(x) != length(y) && length(x) < 20 && length(y) < 20) {
        s1 <- siegel.test(x, y, reverse=FALSE)
        s2 <- siegel.test(x, y, reverse=TRUE)
        if (min(c(sum(s1$rank.x), sum(s1$rank.y))) < min(c(sum(s2$rank.x), sum(s2$rank.y)))){
            s <- s1
        }
        else {
            s <- s2
        }
    }
    else {
        s <- siegel.test(x, y)
    }
    return(s)
}

s <- siegel_tukey(station1, station2)

cat(
    sprintf(
        'Probability that the 2 stations are consistent in playing ads: %.4f\nsd1=%.4f, sd2=%.4f',
        s$p.value,
        sd(station1),
        sd(station2)
    )
)

Loading required package: jmuOutlier


Probability that the 2 stations are consistent in playing ads: 0.6964
sd1=20.4034, sd2=18.9297

In [15]:
# Problem 3
# First sample consists from the number of passangers travelling each week on a certain route.
# Second sample represents the same quantity but for another type of aeroplane (with same number of seats
# as previous model). Is the second model more popular than the first ?
sample1 <- c(3204, 2967, 3053, 3267, 3370, 3492, 3105, 3330)
sample2 <- c(3568, 3299, 3618, 3494)
w <- wilcox.test(sample1, sample2, exact=FALSE)
cat(
    sprintf(
        'There probability of no significant difference between the aeroplane models is: %.4f (rank=%.2f),\nmean1=%.4f, mean2=%.4f', 
        w$p.value,
        w$statistic,
        mean(sample1),
        mean(sample2)
    )
)

There probability of no significant difference between the aeroplane models is: 0.0338 (rank=3.00),
mean1=3223.5000, mean2=3494.7500

In [20]:
# Problem 4:
# The problem consists in determining whether domestic cats are better fed than stray cats. 
# There are 30 samples of weights domestic cats and 15 of stray. The smallest rank total is 234.
# Is there a significant difference in cat weights ?

# Solution
# We apply the formula for the case when one sample has 20 or more measurements:
wilcox2.test <- function(na, nb, R) {
    nr <- min(na, nb)
    z <- sqrt(3) * (nr * (na + nb + 1) - 2 * R) / (sqrt(na*nb*(na + nb + 1)))
    return(pnorm(z, lower.tail=FALSE))
}

cat(sprintf('Probability of no significant difference in weights: %.4f', wilcox2.test(30, 15, 234)))

Probability of no significant difference in weights: 0.0038